# Data Collection

In [1]:
!pip install -U scikit-learn

In [2]:
!pip install sqlalchemy==1.4.46

  Using cached SQLAlchemy-1.4.46-cp311-cp311-win_amd64.whl.metadata (10 kB)
Using cached SQLAlchemy-1.4.46-cp311-cp311-win_amd64.whl (1.6 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.31
    Uninstalling SQLAlchemy-2.0.31:
      Successfully uninstalled SQLAlchemy-2.0.31


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.46 which is incompatible.


In [3]:
!pip install ipython-sql

  Using cached SQLAlchemy-2.0.31-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
Using cached SQLAlchemy-2.0.31-cp311-cp311-win_amd64.whl (2.1 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.46
    Uninstalling SQLAlchemy-1.4.46:
      Successfully uninstalled SQLAlchemy-1.4.46


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [5]:
%load_ext sql

In [6]:
import csv, sqlite3
import requests
import datetime

In [7]:
# print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# print all of the data in a feature
pd.set_option('display.max_colwidth', None)

In [8]:
# read dataset and display the first 5 rows
df = pd.read_csv('social_media_data.csv')
df.head()

,date,platform,post_type,likes,shares,comments,views
0,2023-05-24,Twitter,image,868.0,238.0,352.0,8836.0
1,2023-02-23,Twitter,image,944.0,610.0,146.0,5554.0
2,2023-03-18,Instagram,image,1067.0,858.0,365.0,4018.0
3,2023-06-12,Twitter,text,780.0,427.0,100.0,6999.0
4,2023-02-03,Instagram,text,836.0,893.0,443.0,5802.0


# Data Cleaning

In [9]:
df.dtypes

date          object
platform      object
post_type     object
likes        float64
shares       float64
comments     float64
views        float64
dtype: object

In [10]:
#calculate missing values
df.isnull().sum()/len(df)*100

date         0.0
platform     0.0
post_type    0.0
likes        0.4
shares       0.4
comments     0.4
views        0.5
dtype: float64

In [11]:
#printing the number of missing values based on what was found 
print("number of NaN values for the column likes :", df['likes'].isnull().sum())
print("number of NaN values for the column shares :", df['shares'].isnull().sum())
print("number of NaN values for the column comments :", df['comments'].isnull().sum())
print("number of NaN values for the column views :", df['views'].isnull().sum())

number of NaN values for the column likes : 8
number of NaN values for the column shares : 8
number of NaN values for the column comments : 8
number of NaN values for the column views : 10


In [12]:
#cleaning up the data by filling the missing values with the mean of the column
likes_mean = df['likes'].mean()
df['likes'].replace(np.nan,likes_mean,inplace=True)

shares_mean = df['shares'].mean()
df['shares'].replace(np.nan,shares_mean,inplace=True)

comments_mean = df['comments'].mean()
df['comments'].replace(np.nan,comments_mean,inplace=True)

views_mean = df['views'].mean()
df['views'].replace(np.nan,views_mean,inplace=True)

In [13]:
#checking for missing values
print("number of NaN values for the column likes :", df['likes'].isnull().sum())
print("number of NaN values for the column shares :", df['shares'].isnull().sum())
print("number of NaN values for the column comments :", df['comments'].isnull().sum())
print("number of NaN values for the column views :", df['views'].isnull().sum())

number of NaN values for the column likes : 0
number of NaN values for the column shares : 0
number of NaN values for the column comments : 0
number of NaN values for the column views : 0


In [14]:
df['platform'].value_counts().to_frame()

,count
platform,
Twitter,685
Instagram,678
Facebook,631
NonExistingPlatform,6


##### Since there's a NonExisting Platform, I delete it as it is of no use and it will hinder the data 

In [15]:
df =  df[df['platform']!= 'NonExistingPlatform']

In [16]:
df['platform'].value_counts().to_frame()

,count
platform,
Twitter,685
Instagram,678
Facebook,631


In [17]:
#converting the columns to their appropriate data types
df['likes'] = df['likes'].astype('int64')
df['shares'] = df['shares'].astype('int64')
df['comments'] = df['comments'].astype('int64')
df['views'] = df['views'].astype('int64')

df['date'] = df['date'].astype('datetime64[ns]')

In [18]:
df.dtypes

date         datetime64[ns]
platform             object
post_type            object
likes                 int64
shares                int64
comments              int64
views                 int64
dtype: object

# Data Analysis

In [19]:
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [20]:
%sql sqlite:///my_data1.db

In [21]:
df.to_sql("SOCIALDATA", con, if_exists='replace', index=False,method="multi")

1994

In [22]:
%sql create table SOCIALDATATBL as select * from SOCIALDATA where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SOCIALDATATBL already exists
[SQL: create table SOCIALDATATBL as select * from SOCIALDATA where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### Q1) What was the total number of likes, shares, comments, and views (for videos) over the year?

In [23]:
%sql select sum(likes), sum(shares), sum(comments), sum(views) from SOCIALDATATBL where post_type = 'video'

 * sqlite:///my_data1.db
Done.


sum(likes),sum(shares),sum(comments),sum(views)
815438,311331,194617,4485335


### Q2) Which platform had the highest engagement? (engagement = likes + shares + comments)

In [24]:
#adding likes, shares and comments and name them as total_engagement and then sort the platforms in a descending order
%sql select platform, sum(likes+ shares+ comments) as total_engagement  from SOCIALDATATBL group by platform order by total_engagement desc

 * sqlite:///my_data1.db
Done.


platform,total_engagement
Instagram,1426670
Twitter,1413395
Facebook,1334070


### Q3) What was the average number of likes, shares, comments, and views per post on each platform?

In [25]:
%sql select platform, avg(likes) as avg_likes, avg(shares) as avg_shares, avg(comments) as avg_comments, avg(views) as avg_views from SOCIALDATATBL group by platform 

 * sqlite:///my_data1.db
Done.


platform,avg_likes,avg_shares,avg_comments,avg_views
Facebook,1289.9746434231379,513.0903328050713,311.1505546751189,6984.123613312203
Instagram,1305.787610619469,497.3702064896755,301.07522123893807,6906.141592920354
Twitter,1275.014598540146,482.4802919708029,305.85547445255474,7013.051094890511


### Q4) What was the total number of each type of post (video, image, text) on each platform?



In [26]:
%sql select platform, sum(CASE WHEN post_type = 'video' then 1 else 0 end) AS video, sum(case when post_type = 'image' then 1 else 0 end) AS image, sum(case when post_type = 'text' then 1 else 0 end) AS text from SOCIALDATATBL group by platform

 * sqlite:///my_data1.db
Done.


platform,video,image,text
Facebook,209,218,204
Instagram,208,267,203
Twitter,217,249,219


### Q5) Which type of post had the highest average engagement?

In [32]:
%sql select post_type, likes, shares, comments, avg(COALESCE(likes, 0)+COALESCE(shares,0)+COALESCE(comments,0)) as avg_engagement from SOCIALDATATBL group by post_type order by avg_engagement desc

 * sqlite:///my_data1.db
Done.


post_type,likes,shares,comments,avg_engagement
text,780,427,100,2131.4984025559106
video,515,216,380,2084.205047318612
image,868,238,352,2068.707084468665


### Q6) What was the total engagement in each quarter of the year?

In [51]:
%sql select sum(likes+shares+comments) as Q1_total_engagement from socialdatatbl where date between '2022-10-13' and '2023-01-13'

 * sqlite:///my_data1.db
Done.


Q1_total_engagement
1071642


In [52]:
%sql select sum(likes+shares+comments) as Q2_total_engagement from socialdatatbl where date between '2023-01-14' and '2023-04-14'

 * sqlite:///my_data1.db
Done.


Q2_total_engagement
1042833


In [53]:
%sql select sum(likes+shares+comments) as Q3_total_engagement from socialdatatbl where date between '2023-04-15' and '2023-07-15'

 * sqlite:///my_data1.db
Done.


Q3_total_engagement
991936


In [54]:
%sql select sum(likes+shares+comments) as Q4_total_engagement from socialdatatbl where date between '2023-07-15' and '2023-10-15'

 * sqlite:///my_data1.db
Done.


Q4_total_engagement
1042498
